In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout, BatchNormalization
from keras.layers import ELU, PReLU, LeakyReLU
from keras.optimizers import Adam
from sklearn.metrics import classification_report
import joblib
import os
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import keras as keras

2022-11-18 06:41:15.148443: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 06:41:15.287064: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-18 06:41:15.287094: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-18 06:41:15.317922: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-18 06:41:19.003087: W tensorflow/stream_executor/pla

In [2]:
nn = keras.models.load_model('/home/cdsw/06_ML_MODEL/nn_model')

df = pd.read_csv('/home/cdsw/02_PREDICTION_DATA/processed_data.csv')
print("Raw shape: ", df.shape)

y = df['TARGET']
feats = [f for f in df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
X = df[feats]
print("X shape: ", X.shape)

print("\nPreparing data...")

X = X.fillna(X.mean()).clip(-1e11,1e11)
scaler = MinMaxScaler()
scaler.fit(X)

nn_predict=nn.predict(X).flatten().clip(0,1)

2022-11-18 06:41:25.921354: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-18 06:41:25.921387: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-18 06:41:25.921405: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (avw1vloga5hhciwb): /proc/driver/nvidia/version does not exist
2022-11-18 06:41:25.921829: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Raw shape:  (307507, 721)
X shape:  (307507, 719)     y shape: (307507,)

Preparing data...
9610/9610 [==============================] - 16s 2ms/step


In [9]:
print( 'Saving results...' )
nn_predict_result = pd.DataFrame()
nn_predict_result['SK_ID_CURR'] = df['SK_ID_CURR']
nn_predict_result['TARGET'] = nn_predict


nn_predict_result.to_csv('/home/cdsw/03_PREDICTION_RESULT/nn_predict_result.csv')

Saving results...
